In [21]:
import os
import cv2
import torch
import numpy as np
import pandas as pd

import wandb
import datetime
from tqdm.auto import tqdm

from timm.models import resnet18, resnet50d, resnet101c, efficientnetv2_s, efficientnetv2_m, vit_tiny_r_s16_p8_384, vit_small_r26_s32_384, tresnet_m, tresnet_l, swinv2_cr_tiny_384
from torchsummary import summary
from thop import profile
from utils import set_seed

from models import Tresnet_m_ml_decoder_recipy
torch.cuda.empty_cache()
set_seed(31)

In [2]:
model = tresnet_m(pretrained=False, num_classes=110).cuda()
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
      SpaceToDepth-1           [-1, 48, 56, 56]               0
            Conv2d-2           [-1, 64, 56, 56]          27,648
          Identity-3           [-1, 64, 56, 56]               0
         LeakyReLU-4           [-1, 64, 56, 56]               0
    BatchNormAct2d-5           [-1, 64, 56, 56]             128
       ConvNormAct-6           [-1, 64, 56, 56]               0
            Conv2d-7           [-1, 64, 56, 56]          36,864
          Identity-8           [-1, 64, 56, 56]               0
         LeakyReLU-9           [-1, 64, 56, 56]               0
   BatchNormAct2d-10           [-1, 64, 56, 56]             128
      ConvNormAct-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
         Identity-13           [-1, 64, 56, 56]               0
         Identity-14           [-1, 64,

In [3]:
dummy_input = torch.randn(1, 3, 224, 224).cuda()

flops, params = profile(model, inputs=(dummy_input,))

print(f"Total Flops: {flops}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Total Flops: 5733267968.0


In [4]:
from torch.profiler import record_function, ProfilerActivity

def train_and_inference():
    # 학습
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = torch.nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for _ in range(10):  # 예시로 10 에폭을 수행합니다.
        optimizer.zero_grad()
        output = model(dummy_input)
        loss = criterion(output, torch.tensor([0]).to(device))
        loss.backward()
        optimizer.step()

    # 추론
    model.eval()
    with torch.no_grad():
        for _ in range(100):  # 예시로 100번 추론을 수행합니다.
            output = model(dummy_input)

# PyTorch Profiler로 학습 및 추론 시간 측정
with torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        train_and_inference()

# 결과 출력
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

STAGE:2023-07-24 12:43:26 30385:30385 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-24 12:43:31 30385:30385 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-24 12:43:31 30385:30385 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        29.57%        1.456s        86.88%        4.279s        4.279s       0.000us         0.00%     501.406ms     501.406ms             1  
                                           aten::conv2d         1.51%      74.244ms        25.12%        1.237s     113.593us       0.000us         0.00%     343.880ms      31.578us         10890  
         

In [5]:
torch.cuda.empty_cache()
dummy_input = torch.randn(1, 3, 224, 224).cuda()

In [6]:
model = Tresnet_m_ml_decoder_recipy(num_classes=110).cuda()
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
      SpaceToDepth-1           [-1, 48, 56, 56]               0
            Conv2d-2           [-1, 64, 56, 56]          27,648
          Identity-3           [-1, 64, 56, 56]               0
         LeakyReLU-4           [-1, 64, 56, 56]               0
    BatchNormAct2d-5           [-1, 64, 56, 56]             128
       ConvNormAct-6           [-1, 64, 56, 56]               0
            Conv2d-7           [-1, 64, 56, 56]          36,864
          Identity-8           [-1, 64, 56, 56]               0
         LeakyReLU-9           [-1, 64, 56, 56]               0
   BatchNormAct2d-10           [-1, 64, 56, 56]             128
      ConvNormAct-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
         Identity-13           [-1, 64, 56, 56]               0
         Identity-14           [-1, 64,

In [7]:
flops, params = profile(model, inputs=(dummy_input,))

print(f"Total Flops: {flops}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Total Flops: 6125607424.0


In [8]:
def train_and_inference():
    # 학습
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = torch.nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for _ in range(10):  # 예시로 10 에폭을 수행합니다.
        optimizer.zero_grad()
        output = model(dummy_input)
        loss = criterion(output, torch.tensor([0]).to(device))
        loss.backward()
        optimizer.step()

    # 추론
    model.eval()
    with torch.no_grad():
        for _ in range(100):  # 예시로 100번 추론을 수행합니다.
            output = model(dummy_input)

# PyTorch Profiler로 학습 및 추론 시간 측정
with torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        train_and_inference()

# 결과 출력
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

STAGE:2023-07-24 12:44:58 30385:30385 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-24 12:45:07 30385:30385 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-24 12:45:07 30385:30385 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        34.32%        2.954s        83.34%        7.174s        7.174s       0.000us         0.00%     597.306ms     597.306ms             1  
                                           aten::conv2d         0.82%      70.241ms        14.55%        1.252s     114.993us       0.000us         0.00%     321.900ms      29.559us         10890  
         

In [9]:
torch.cuda.empty_cache()
dummy_input = torch.randn(1, 3, 224, 224).cuda()

In [10]:
model = efficientnetv2_s(pretrained=False, num_classes=110).cuda()
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 112, 112]             648
          Identity-2         [-1, 24, 112, 112]               0
              SiLU-3         [-1, 24, 112, 112]               0
    BatchNormAct2d-4         [-1, 24, 112, 112]              48
            Conv2d-5         [-1, 24, 112, 112]           5,184
          Identity-6         [-1, 24, 112, 112]               0
              SiLU-7         [-1, 24, 112, 112]               0
    BatchNormAct2d-8         [-1, 24, 112, 112]              48
          Identity-9         [-1, 24, 112, 112]               0
        ConvBnAct-10         [-1, 24, 112, 112]               0
           Conv2d-11         [-1, 24, 112, 112]           5,184
         Identity-12         [-1, 24, 112, 112]               0
             SiLU-13         [-1, 24, 112, 112]               0
   BatchNormAct2d-14         [-1, 24, 1

In [11]:
flops, params = profile(model, inputs=(dummy_input,))

print(f"Total Flops: {flops}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Total Flops: 2848820672.0


In [12]:
def train_and_inference():
    # 학습
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = torch.nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for _ in range(10):  # 예시로 10 에폭을 수행합니다.
        optimizer.zero_grad()
        output = model(dummy_input)
        loss = criterion(output, torch.tensor([0]).to(device))
        loss.backward()
        optimizer.step()

    # 추론
    model.eval()
    with torch.no_grad():
        for _ in range(100):  # 예시로 100번 추론을 수행합니다.
            output = model(dummy_input)

# PyTorch Profiler로 학습 및 추론 시간 측정
with torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        train_and_inference()

# 결과 출력
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

STAGE:2023-07-24 12:47:23 30385:30385 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-24 12:47:31 30385:30385 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-24 12:47:31 30385:30385 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        24.32%        1.873s        89.79%        6.914s        6.914s       0.000us         0.00%     445.900ms     445.900ms             1  
                                           aten::conv2d         1.66%     127.779ms        28.07%        2.161s     115.581us       0.000us         0.00%     249.582ms      13.347us         18700  
         

In [13]:
torch.cuda.empty_cache()
dummy_input = torch.randn(1, 3, 384, 384).cuda()

In [22]:
model = vit_small_r26_s32_384(pretrained=False, num_classes=110).cuda()
summary(model, (3, 384, 384))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
     StdConv2dSame-1         [-1, 64, 192, 192]           9,408
          Identity-2         [-1, 64, 192, 192]               0
              ReLU-3         [-1, 64, 192, 192]               0
      GroupNormAct-4         [-1, 64, 192, 192]             128
     MaxPool2dSame-5           [-1, 64, 96, 96]               0
     StdConv2dSame-6          [-1, 256, 96, 96]          16,384
          Identity-7          [-1, 256, 96, 96]               0
          Identity-8          [-1, 256, 96, 96]               0
      GroupNormAct-9          [-1, 256, 96, 96]             512
   DownsampleConv-10          [-1, 256, 96, 96]               0
    StdConv2dSame-11           [-1, 64, 96, 96]           4,096
         Identity-12           [-1, 64, 96, 96]               0
             ReLU-13           [-1, 64, 96, 96]               0
     GroupNormAct-14           [-1, 64,

In [23]:
flops, params = profile(model, inputs=(dummy_input,))

print(f"Total Flops: {flops}")

[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Total Flops: 3197737728.0


In [24]:
def train_and_inference():
    # 학습
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = torch.nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for _ in range(10):  # 예시로 10 에폭을 수행합니다.
        optimizer.zero_grad()
        output = model(dummy_input)
        loss = criterion(output, torch.tensor([0]).to(device))
        loss.backward()
        optimizer.step()

    # 추론
    model.eval()
    with torch.no_grad():
        for _ in range(100):  # 예시로 100번 추론을 수행합니다.
            output = model(dummy_input)

# PyTorch Profiler로 학습 및 추론 시간 측정
with torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        train_and_inference()

# 결과 출력
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

STAGE:2023-07-24 13:05:01 30385:30385 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-24 13:05:06 30385:30385 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-24 13:05:06 30385:30385 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        28.17%        1.390s        89.08%        4.396s        4.396s       0.000us         0.00%     794.841ms     794.841ms             1  
                                           aten::linear         1.01%      50.021ms        12.74%     628.947ms     116.688us       0.000us         0.00%     148.254ms      27.505us          5390  
         

In [17]:
torch.cuda.empty_cache()
dummy_input = torch.randn(1, 3, 384, 384).cuda()

In [18]:
model = swinv2_cr_tiny_384(pretrained=False, num_classes=110).cuda()
summary(model, (3, 384, 384))

/opt/conda/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 96, 96]           4,704
         LayerNorm-2           [-1, 96, 96, 96]             192
        PatchEmbed-3           [-1, 96, 96, 96]               0
          Identity-4           [-1, 96, 96, 96]               0
            Linear-5             [-1, 144, 288]          27,936
            Linear-6                  [-1, 384]           1,152
              ReLU-7                  [-1, 384]               0
           Dropout-8                  [-1, 384]               0
            Linear-9                    [-1, 3]           1,155
          Dropout-10                    [-1, 3]               0
              Mlp-11                    [-1, 3]               0
          Dropout-12          [-1, 3, 144, 144]               0
           Linear-13              [-1, 144, 96]           9,312
          Dropout-14              [-1, 

In [19]:
flops, params = profile(model, inputs=(dummy_input,))

print(f"Total Flops: {flops}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
Total Flops: 14116601088.0


In [20]:
def train_and_inference():
    # 학습
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = torch.nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for _ in range(10):  # 예시로 10 에폭을 수행합니다.
        optimizer.zero_grad()
        output = model(dummy_input)
        loss = criterion(output, torch.tensor([0]).to(device))
        loss.backward()
        optimizer.step()

    # 추론
    model.eval()
    with torch.no_grad():
        for _ in range(100):  # 예시로 100번 추론을 수행합니다.
            output = model(dummy_input)

# PyTorch Profiler로 학습 및 추론 시간 측정
with torch.profiler.profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        train_and_inference()

# 결과 출력
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

STAGE:2023-07-24 12:50:05 30385:30385 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-07-24 12:50:11 30385:30385 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-07-24 12:50:11 30385:30385 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference        28.95%        1.656s        88.01%        5.035s        5.035s       0.000us         0.00%        1.524s        1.524s             1  
                                           aten::linear         0.91%      52.042ms        16.76%     958.566ms     114.661us       0.000us         0.00%     955.566ms     114.302us          8360  
         

In [ ]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from dataset import *

In [ ]:
tf = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(),
    A.Normalize(),
])

train_dataset = CustomDataset(
            "/opt/ml/level3_cv_finalproject-cv-01/model", is_train=True, tf=tf
        )

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=4,
            shuffle=True,
            num_workers=8,
            drop_last=True,
        )

In [ ]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score


cls_criterion = nn.CrossEntropyLoss()
rcp_criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels, ingredients in train_loader:
        images, labels, ingredients = (
                images.clone().detach().cuda(),
                labels.clone().detach().cuda(),
                ingredients.clone().detach().cuda(),
            )
        print(f"image : {images.shape}")
        print(f"label : {labels.shape}")
        print(f"ingredient : {ingredients.shape}")
        print(f"max : {torch.max(images)}, min : {torch.min(images)}")

        cls_output, rcp_output = model(images)
        print(f"cls_output : {cls_output.shape}")
        print(f"rcp_output : {rcp_output.shape}")
        print(f"max : {torch.max(cls_output)}, min : {torch.min(cls_output)}")
        print(f"max : {torch.max(rcp_output)}, min : {torch.min(rcp_output)}")

        cls_output = F.sigmoid(cls_output)
        rcp_output = F.sigmoid(rcp_output)
        print(f"max : {torch.max(cls_output)}, min : {torch.min(cls_output)}")
        print(f"max : {torch.max(rcp_output)}, min : {torch.min(rcp_output)}")
        print(f"max : {torch.max(labels)}, min : {torch.min(labels)}")
        print(f"max : {torch.max(ingredients)}, min : {torch.min(ingredients)}")

        cls_loss = cls_criterion(cls_output, labels)
        rcp_loss = rcp_criterion(rcp_output, ingredients)
        print(f"Loss : {cls_loss}, {rcp_loss}")

        cls_thr = 0.5
        rcp_thr = 0.5
        cls_output = (cls_output >= cls_thr).float()
        rcp_output = (rcp_output >= cls_thr).float()
        cls_loss = cls_criterion(cls_output, labels)
        rcp_loss = rcp_criterion(rcp_output, ingredients)
        print(f"Loss : {cls_loss}, {rcp_loss}")

        cls_f1 = f1_score(y_pred=cls_output.cpu(), y_true=labels.cpu(), average="macro", zero_division=0)
        rcp_f1 = f1_score(y_pred=rcp_output.cpu(), y_true=ingredients.cpu(), average="macro", zero_division=0)
        cls_precision = precision_score(y_pred=cls_output.cpu(), y_true=labels.cpu(), average="macro", zero_division=0)
        rcp_precision = precision_score(y_pred=rcp_output.cpu(), y_true=ingredients.cpu(), average="macro", zero_division=0)
        print(f"F1-Score : {cls_f1}, {rcp_f1}")
        print(f"Precision Score : {cls_precision}, {rcp_precision}")
        raise
        for i in range(images.shape[0]):
            image = images[i].cpu()
            image = image.permute(1, 2, 0)
            image_np = image.numpy()
            print(train_dataset.classes[labels[i].item()])
            plt.imshow(image_np)
            plt.axis('off')
            plt.show()
        raise

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch import cuda
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
model.to("cuda" if cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
max_epoch = 50
scheduler = CosineAnnealingLR(optimizer, T_max=max_epoch, eta_min=1e-3)

In [ ]:
lrs = []
for epoch in range(max_epoch):
    optimizer.zero_grad()
    optimizer.step()
    scheduler.step()
    current_lr = optimizer.param_groups[0]["lr"]
    lrs.append(current_lr)
fig, ax = plt.subplots()
ax.plot(lrs)
plt.grid()
plt.show()

In [ ]:
import json
import os

path = "/opt/ml/level3_cv_finalproject-cv-01/model/data/json/BBQ"
file_names = os.listdir(path)
for file_name in file_names:
    # new_name = file_name.replace("onirigi", "onigiri")
    with open(os.path.join(path, file_name)) as file:
        data = json.load(file)
    data['food_class'] = ["bbq"]
    with open(os.path.join("/opt/ml/level3_cv_finalproject-cv-01/model/BBQ", file_name), "w") as file:
        json.dump(data, file)
    